In [9]:
import pandas as pd
import re
import os
import time
import lib
from textwrap import dedent
from googletrans import Translator
from tqdm.auto import tqdm
translator = Translator()


In [8]:
import os
os.environ["HF_ENDPOINT"] = "https://huggingface.co"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Doogie/waynehills_sentimental_kor")
model = AutoModelForSequenceClassification.from_pretrained("Doogie/waynehills_sentimental_kor")

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/815 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0M [00:00<?, ?B/s]

In [10]:
def cleaning_korean(content):
    content = content.replace('\n\n','')
    content = ' '.join(content.split(' '))

    #클리닝1
    content = re.sub(r'... 기자', '', content)
    content = re.sub(r'...기자', '', content)

    #괄호안 제거 
    ''' 괄호안 내용 제거하기 '''
    pattern_bracket1 = r'\([^)]*\)'
    pattern_bracket2 = r'\{[^)]*\}'
    pattern_bracket3 = r'\<[^)]*\>'
    pattern_bracket4 = r'\[[^)]*\]'

    x = '이건 {괄호 안의 불필요한 정보를} 삭제하는 코드다.'
    content = re.sub(pattern=pattern_bracket1, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket2, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket3, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket4, repl='', string= content)
    content = ' '.join(content.split())
    #클리닝
    content = content.lower() #lower case
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    content = re.sub(pattern=pattern, repl='', string=content)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    content = re.sub(pattern=pattern, repl='', string=content)
    # 특수기호 특수
    content = re.sub('[-=+#/\?:^$@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》Δ△·“=◎<>▷]', '', content)
    content = re.sub('[一-龥]', '',content)
    #pattern = '[^\w\s]'         # 특수기호제거
    #content = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"]', '',str(content)) #remove punctuation
    content = re.sub(r"[^a-zA-Z0-9가-힣\.\s]","",content)
    content = re.sub(pattern=pattern, repl='', string=content)
    content = re.sub(r'\s+', ' ', content) #remove extra space
    content = re.sub(r'<[^>]+>','',content) #remove Html tags
    content = re.sub(r'\s+', ' ', content) #remove spaces
    content = re.sub(r"^\s+", '', content) #remove space from start
    content = re.sub(r'\s+$', '', content) #remove space from the end
    return content

In [11]:
df = pd.read_excel('./scrape_oci.xlsx', index_col=0)

In [12]:
all_contents = []
for i, row in df.iterrows():
    tit = row.Titles
    cot = row.Contents
    cot_all = str(tit) + '.' + str(cot)
    all_contents.append(cleaning_korean(cot_all))

In [ ]:
%%time
contents_en = []
for x in tqdm(all_contents):
    try:
        src = "다음의 문장이 긍정,중립,부정 중 어느 카테고리에 속하는지 알려줘 " + '"' + str(x) + '"'
        src_en = translator.translate(src).text
        contents_en.append(src_en)
    except:
        contents_en.append('중립')

In [22]:
tt = tokenizer(all_contents[:3], return_tensors="pt", padding=True)

In [24]:
model(**tt)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.7370, -0.7105],
        [ 0.0555, -0.0207],
        [ 1.3740, -1.3492]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
all_contents[:3]

['중국 신장지구에 지진 진도7.3.냉무',
 '신한 .혼자 눌러대느라 애쓴다 공매상환 하려면 못오르게 눌러야지 그렇지만 그것도 곧한계가 온다 너같은 공매들땜에 일반주주들 손해으니 이제적당히 해라 배불르지 않냐',
 '애쓴다 비싸게.팔려고 저 매수호가 봐라']